In [1]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath('..'))

from src.data_loader import DataLoader
from src.risk_factors import PortfolioMetrics
from src.var_modules import RiskCalculator
from src.stress_scenarios import StressTester

In [2]:
# Initialisation rapide
TICKERS = ['AAPL', 'JPM', 'XOM', 'TLT']
WEIGHTS = [0.4, 0.2, 0.2, 0.2]
returns = DataLoader(TICKERS, "2020-01-01", "2025-12-31").calculate_returns(DataLoader(TICKERS, "2020-01-01", "2025-12-31").get_data())
calc = RiskCalculator(PortfolioMetrics(returns, WEIGHTS))

# Création d'une liste pour stocker les résultats
summary_data = []

# 1. Méthodes Classiques
vp, cp = calc.parametric_var_cvar()
vh, ch = calc.historical_var_cvar()
summary_data.append(["Paramétrique (Normale)", vp, cp, "Théorique"])
summary_data.append(["Historique (Réelle)", vh, ch, "Basé sur le passé"])

# 2. Stress Tests
scenarios = StressTester.get_scenarios()
for name, factor in scenarios.items():
    v_mc, c_mc, _ = calc.monte_carlo_var_cvar(stress_factor=factor)
    summary_data.append([f"Monte Carlo ({name})", v_mc, c_mc, f"Volatilité x{factor}"])

# Création du DataFrame
df_results = pd.DataFrame(summary_data, columns=["Méthode / Scénario", "VaR 95%", "CVaR 95%", "Note"])

# Formatage en pourcentage pour l'affichage
df_display = df_results.copy()
df_display["VaR 95%"] = df_display["VaR 95%"].apply(lambda x: f"{x:.2%}")
df_display["CVaR 95%"] = df_display["CVaR 95%"].apply(lambda x: f"{x:.2%}")

display(df_display)

Chargement des données pour ['AAPL', 'JPM', 'XOM', 'TLT']...


[*********************100%***********************]  4 of 4 completed


,Méthode / Scénario,VaR 95%,CVaR 95%,Note
0,Paramétrique (Normale),1.99%,2.51%,Théorique
1,Historique (Réelle),1.73%,2.93%,Basé sur le passé
2,Monte Carlo (Normal),2.01%,2.55%,Volatilité x1.0
3,Monte Carlo (Crise_Moderee),2.48%,3.15%,Volatilité x1.25
4,Monte Carlo (Crise_Majeure),3.01%,3.79%,Volatilité x1.5
5,Monte Carlo (Black_Swan),4.11%,5.21%,Volatilité x2.0
